In [ ]:
import pandas as pd
import numpy as np
import feather
import datetime

import geopandas as gpd
import PyNergy as pn

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

energy = pd.read_feather('data/energy.ftr')
energy = energy[ (energy['date'].dt.year < 2021) & (energy['date'].dt.year > 2012)]

sns.set_style = "seaborn-whitegrid"

color = "#333"
bck_color = "#fff"
grid_color = "#ddd"

dark_mode = True
if dark_mode:
    color = "#eee"
    bck_color = "#333"
    grid_color = "#666"


charts_colors = {
    "blue_light": "#26ffff",
    "blue": "#2696ff",
    "blue_dark": "#0759a6",

    "pink_light": "#fdc2ff",
    "pink": "#f830ff",
    "pink_dark": "#830087",

    "orange_light": "#ffdc8c",
    "orange": "#ffc130",
    "orange_dark": "#de9a00",

    "yellow_light": "#fcffcc",
    "yellow": "#ffff00",
    "yellow_dark": "#8f8f09",

    "green_light": "#abffa6",
    "green": "#33ff26",
    "green_dark": "#157a0f",

    "red_light": "#ff7878",
    "red": "#ED0000",
    "red_dark": "#a11515"   
}
    

sns.set(
    rc={
        "axes.facecolor": bck_color,
        "grid.color": grid_color,
        "grid.linestyle": "-",
        "figure.facecolor": bck_color,
        "text.color": color,
        "xtick.color": color,
        "ytick.color": color,
        "axes.labelcolor": color,
        "axes.titlecolor": color,
        "axes.grid": True,
        'axes.spines.left': True,
        'axes.spines.bottom': True,
        'axes.spines.right': False,
        'axes.spines.top': False,
        "axes.edgecolor": grid_color,
    }
)

energy['date'] = pd.to_datetime(energy['date'], utc=True)


In [ ]:
energy['total'] = energy['nucleaire'] + energy['thermique'] + energy['hydraulique'] + energy['eolien'] + energy['solaire'] + energy['bioenergies']
consommation = energy[['date', 'consommation', 'total']]
energy.drop(columns=['consommation', 'pompage', 'ech_physiques'], inplace=True)


In [ ]:
prod_by_year = energy.groupby(pd.Grouper(key='date', freq='Y'))['total'].sum().reset_index()

prod_by_year['total'] = prod_by_year['total'] / 1000

fig, ax = plt.subplots(figsize=(10, 6))
g = sns.barplot(
    x=prod_by_year['date'].dt.year, 
    y='total', data=prod_by_year, ax=ax, 
    color=charts_colors['blue'], edgecolor=grid_color)

ax.set_title('Production selon les années en gW')
ax.set_xlabel('Années')
ax.set_ylabel('Production')

ax.set_yticks(np.arange(100, 1100, 100))
plt.savefig("outputs/03.2.1_-_Production_par_annee.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
prod_by_month = energy.copy()

prod_by_month['m'] = prod_by_month['date'].dt.month
prod_by_month['mn'] = prod_by_month['date'].dt.month_name()
prod_by_month = prod_by_month.groupby(['m', 'mn'])['total'].mean().reset_index()
prod_by_month.sort_values(by='m', ascending=True, inplace=True)

fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(
    x='mn', 
    y='total', 
    data=prod_by_month, ax=ax,
    color=charts_colors['blue'], linewidth=4)

ax.set_title('Production selon les mois en gW')
ax.set_xlabel('Mois')
ax.set_ylabel('Production')

plt.setp(ax.get_xticklabels(), rotation=45)
plt.savefig("outputs/03.2.2_-_Production_par_mois.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
prod_by_dow = energy.copy()
prod_by_dow['dow'] = prod_by_dow['date'].dt.dayofweek
prod_by_dow['dn'] = prod_by_dow['date'].dt.day_name()
prod_by_dow = prod_by_dow.groupby(['dn', 'dow'])['total'].mean().reset_index()
prod_by_dow.sort_values(by='dow', ascending=True, inplace=True)

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(x='dn', y='total', data=prod_by_dow, ax=ax, linewidth=4, color=charts_colors['blue'])

ax.set_title('Production selon les jours de la semaine en mW')
ax.set_xlabel('Jours de la semaine')
ax.set_ylabel('Production')

plt.setp(ax.get_xticklabels(), rotation=45)
plt.savefig("outputs/03.2.3_-_Production_par_jour.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
prod_by_hour = energy.groupby(pd.Grouper(key='date', freq='H'))['total'].mean().reset_index()

month_less_prod = prod_by_hour[ prod_by_hour['date'].dt.month.isin([1, 12])].groupby(pd.Grouper(key='date', freq='H'))['total'].mean().reset_index()
month_most_prod = prod_by_hour[ prod_by_hour['date'].dt.month.isin([7, 8])].groupby(pd.Grouper(key='date', freq='H'))['total'].mean().reset_index()

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(
    x=prod_by_hour['date'].dt.hour, y='total', data=prod_by_hour, ci=None,
    ax=ax, color=charts_colors['yellow'], drawstyle='steps-post', label='Moyenne annuelle')

sns.lineplot(
    x=month_less_prod['date'].dt.hour, y='total', data=month_less_prod, ci=None,
    ax=ax, color=charts_colors['blue_light'], alpha=0.5, drawstyle='steps-post', label='Moyenne hivernale')

sns.lineplot(
    x=month_most_prod['date'].dt.hour, y='total', data=month_most_prod, ci=None,
    ax=ax, color=charts_colors['pink'], alpha=0.5, drawstyle='steps-post', label='Moyenne estivale')

ax.set_title('Production selon les heures de la journée en mW')
ax.set_xlabel('Heures')
ax.set_ylabel('Production')

ax.set_xticks(np.arange(0, 24, 1))

plt.savefig("outputs/03.2.4_-_Production_par_heure.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
prod = energy[ (energy['date'].dt.year < 2021) & (energy['date'].dt.year > 2012) ]
prod = prod.groupby(['code_insee_region', 'region'])['total'].mean().reset_index()

gdf = pn.get_geometry(prod)

c = mcolors.ColorConverter().to_rgb
cmap = pn.make_colormap(
    [c(charts_colors['blue_light']), c(charts_colors['blue_dark'])])

fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot('total', edgecolor=bck_color, legend=True, cmap=cmap, legend_kwds={'shrink': 0.5}, ax=ax)
ax.set_title('Production par région')
ax.axis('off')
plt.savefig("outputs/03.1.5_-_Carte_consommation_par_region.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
consommation['mois'] = consommation.date.dt.month
hours = consommation.groupby([consommation.date.dt.month_name(), 'mois'])[['consommation', 'total']].mean().reset_index()
hours.sort_values(by='mois', ascending=True, inplace=True)
hours = hours.set_index('date')
hours = hours.drop(columns=['mois'])

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(
    data=hours, markers=True, dashes=False, 
    ax=ax, palette=[charts_colors['yellow'], charts_colors['blue']],
    markeredgecolor=bck_color, markeredgewidth=.5, markersize=7.5)

ax.set_title('Consommation moyenne - Production moyenne')
ax.set_xlabel('')

plt.setp(ax.get_xticklabels(), rotation=45)

plt.legend(labels=['Consommation', 'Production'])

plt.savefig("outputs/03.2.5_-_Production_vs_conso_mois.png", dpi=150, bbox_inches='tight')

plt.show()